In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from typing import Callable, List, Tuple

In [2]:
data = pd.read_csv('data/train.csv', parse_dates=['Date'])
stores = pd.read_csv('data/store.csv')
store_features = stores[['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2']]

data = \
(
    data
    .merge(store_features.set_index('Store'), on='Store', how='left')  # Add store features
    .loc[data.Open == 1]  # We don't need closed stores data
    .drop(columns=['Open'])
    .sort_values('Date', ascending=True)  # We'll need our values sorted for later
)
data.head(5)

/Users/anvil/envs/rossman/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2
1017190,1097,2,2013-01-01,5961,1405,0,a,1,b,b,720.0,0
1016179,85,2,2013-01-01,4220,619,0,a,1,b,a,1870.0,0
1016353,259,2,2013-01-01,6851,1444,0,a,1,b,b,210.0,0
1016356,262,2,2013-01-01,17267,2875,0,a,1,b,a,1180.0,0
1016368,274,2,2013-01-01,3102,729,0,a,1,b,b,3640.0,1


In [3]:
test_border = '2015-06-17'
val_border = '2015-05-01'

In [4]:
stats = ['mean', 'std', 'min', 'max']

data['month'] = data.Date.dt.month
#data['months_from_start'] = 12 * (data.Date.dt.year - data.Date.min().year) +\
#                            (data.Date.dt.month - data.Date.min().month)

data[['DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'Promo2', 'month']] =\
data[['DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'Promo2', 'month']].astype('category')
data.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,month
1017190,1097,2,2013-01-01,5961,1405,0,a,1,b,b,720.0,0,1
1016179,85,2,2013-01-01,4220,619,0,a,1,b,a,1870.0,0,1
1016353,259,2,2013-01-01,6851,1444,0,a,1,b,b,210.0,0,1
1016356,262,2,2013-01-01,17267,2875,0,a,1,b,a,1180.0,0,1
1016368,274,2,2013-01-01,3102,729,0,a,1,b,b,3640.0,1,1


In [5]:
def merge_last_statistics(df_from: pd.DataFrame, df_to: pd.DataFrame, on_column: str, cols=None) -> pd.DataFrame:
    ''' Returns @df_to with merged unique @df_from columns '''
    # Select only unique columns if not specified explicitly
    cols = cols if cols is not None else df_from.columns.difference(df_to.columns)
    result = \
    (
        df_to
        .merge(
            df_from
                .groupby(on_column, as_index=True)  # as_index=True for an upcoming merge
                [cols]
                .last(),  # Leave only the last entry. It's assumed that df_from is in correct order
            on=on_column,
            how='left'
        )
    )
    return result

In [6]:
def calculate_statistics_naive(data: pd.DataFrame, stats: List[str]) -> pd.DataFrame:
    ''' Implements first (naive) method of adding statistics to the data '''
    data = data.copy()
    for s in stats:
        data[s + '_sales'] = data.groupby('Store').Sales.transform(s)
        data[s + '_customers'] = data.groupby('Store').Customers.transform(s)
    return data

In [7]:
def calculate_statistics_no_leak(data: pd.DataFrame, stats: List[str]) -> pd.DataFrame:
    ''' Implements second (no direct test -> train leak) method '''
    data = data.copy()
    
    sales_cols = [s + '_sales' for s in stats]
    customers_cols = [s + '_customers' for s in stats]
    
    for s in stats:
        data[s + '_sales'] = data.groupby('Store').Sales.transform(
            lambda x: x.shift().expanding().aggregate(s)
        )
        data[s + '_customers'] = data.groupby('Store').Customers.transform(
            lambda x: x.shift().expanding().aggregate(s)
        )
            
    return data

In [8]:
# Zero sales data was ignored in the competition, so it would just make noise for our simple model
data = data.loc[data.Sales != 0]

train = data.loc[data.Date <= val_border]
validation = data.loc[(data.Date > val_border) & (data.Date < test_border)]
test = data.loc[data.Date >= test_border]

In [9]:
# train = calculate_statistics_no_leak(train, stats)
train = calculate_statistics_naive(train, stats)

validation = merge_last_statistics(train.sort_values('Date'), validation, 'Store')
test = merge_last_statistics(train.sort_values('Date'), test, 'Store')

train = train.drop(columns=['Customers'])
validation = validation.drop(columns=['Customers'])

# That is to preserve train order of columns after merging them into val/test
validation = validation[train.columns]
test = test[train.columns]

In [10]:
def make_feval(f: Callable, name: str = None, higher_better: bool = False) -> Callable:
    """
    Function factory to transform @f to @feval required by LightGBM
    Args:
        f: function of 2 arguments (predictions, true_values) -> score
        name: name of function (f.name will be used if None)
        higher_better: True if higher score is better, otherwise False
    Returns:
        feval: function of 2 arguments (predictions, Dataset with true labels) -> (name, score, higher_better)
    """

    def feval(X: np.ndarray, Y: lgb.Dataset):
        return name if name is not None else f.__name__,\
               f(X, Y.get_label()),\
               higher_better

    return feval

def RMSPE(X: np.ndarray, Y: np.ndarray):
    return np.sqrt(np.mean(np.square(((Y - X) / Y))))

In [11]:
validation_set = lgb.Dataset(validation.drop(columns=['Sales', 'Date']), validation['Sales'])
train_set = lgb.Dataset(train.drop(columns=['Sales', 'Date']), train['Sales'])
test_set = test.drop(columns=['Date'])

params = {
    'objective': 'poisson',
    'metric': 'mae',
    'num_iterations': 5000,
    'early_stopping_round': 50
}

model = lgb.train(
    params,
    train_set=train_set,
    valid_sets=[validation_set],
    feval=make_feval(RMSPE),
)

/Users/anvil/envs/rossman/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/anvil/envs/rossman/lib/python3.9/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015809 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2589
[LightGBM] [Info] Number of data points in the train set: 759868, number of used features: 18
[LightGBM] [Info] Start training from score 8.843374
[1]	valid_0's l1: 2158.68	valid_0's RMSPE: 0.501705
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 2076.94	valid_0's RMSPE: 0.4836
[3]	valid_0's l1: 1999.9	valid_0's RMSPE: 0.466509
[4]	valid_0's l1: 1927.67	valid_0's RMSPE: 0.450356
[5]	valid_0's l1: 1859.51	valid_0's RMSPE: 0.435078
[6]	valid_0's l1: 1795.73	valid_0's RMSPE: 0.420705
[7]	valid_0's l1: 1736.1	valid_0's RMSPE: 0.407114
[8]	valid_0's l1: 1679.47	valid_0's RMSPE: 0.394395
[9]	valid_0's l1: 1626.89	valid_0's RMSPE: 0.38238


/Users/anvil/envs/rossman/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/anvil/envs/rossman/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[10]	valid_0's l1: 1578.54	valid_0's RMSPE: 0.371137
[11]	valid_0's l1: 1532.11	valid_0's RMSPE: 0.360531
[12]	valid_0's l1: 1488.8	valid_0's RMSPE: 0.35063
[13]	valid_0's l1: 1448.75	valid_0's RMSPE: 0.341275
[14]	valid_0's l1: 1411.85	valid_0's RMSPE: 0.332632
[15]	valid_0's l1: 1377.89	valid_0's RMSPE: 0.324503
[16]	valid_0's l1: 1346.53	valid_0's RMSPE: 0.316685
[17]	valid_0's l1: 1317.69	valid_0's RMSPE: 0.309584
[18]	valid_0's l1: 1288.62	valid_0's RMSPE: 0.302555
[19]	valid_0's l1: 1263.52	valid_0's RMSPE: 0.296325
[20]	valid_0's l1: 1240.1	valid_0's RMSPE: 0.290148
[21]	valid_0's l1: 1217.54	valid_0's RMSPE: 0.284531
[22]	valid_0's l1: 1197.15	valid_0's RMSPE: 0.279875
[23]	valid_0's l1: 1177.89	valid_0's RMSPE: 0.274495
[24]	valid_0's l1: 1160.75	valid_0's RMSPE: 0.26974
[25]	valid_0's l1: 1144.5	valid_0's RMSPE: 0.265351
[26]	valid_0's l1: 1128.75	valid_0's RMSPE: 0.260952
[27]	valid_0's l1: 1113.76	valid_0's RMSPE: 0.256715
[28]	valid_0's l1: 1100.59	valid_0's RMSPE: 0.25321

[175]	valid_0's l1: 859.369	valid_0's RMSPE: 0.165631
[176]	valid_0's l1: 859.06	valid_0's RMSPE: 0.165165
[177]	valid_0's l1: 858.984	valid_0's RMSPE: 0.165121
[178]	valid_0's l1: 858.419	valid_0's RMSPE: 0.16496
[179]	valid_0's l1: 858.1	valid_0's RMSPE: 0.164504
[180]	valid_0's l1: 858.191	valid_0's RMSPE: 0.164502
[181]	valid_0's l1: 857.711	valid_0's RMSPE: 0.164334
[182]	valid_0's l1: 857.478	valid_0's RMSPE: 0.163823
[183]	valid_0's l1: 856.882	valid_0's RMSPE: 0.1637
[184]	valid_0's l1: 856.267	valid_0's RMSPE: 0.163579
[185]	valid_0's l1: 856.161	valid_0's RMSPE: 0.163562
[186]	valid_0's l1: 856.08	valid_0's RMSPE: 0.163546
[187]	valid_0's l1: 855.397	valid_0's RMSPE: 0.163389
[188]	valid_0's l1: 854.769	valid_0's RMSPE: 0.163202
[189]	valid_0's l1: 854.755	valid_0's RMSPE: 0.163206
[190]	valid_0's l1: 854.518	valid_0's RMSPE: 0.163185
[191]	valid_0's l1: 854.052	valid_0's RMSPE: 0.163023
[192]	valid_0's l1: 854.064	valid_0's RMSPE: 0.16301
[193]	valid_0's l1: 854.05	valid_0's

[341]	valid_0's l1: 830.588	valid_0's RMSPE: 0.150244
[342]	valid_0's l1: 830.426	valid_0's RMSPE: 0.150229
[343]	valid_0's l1: 830.404	valid_0's RMSPE: 0.150229
[344]	valid_0's l1: 830.27	valid_0's RMSPE: 0.150207
[345]	valid_0's l1: 830.111	valid_0's RMSPE: 0.150103
[346]	valid_0's l1: 830.09	valid_0's RMSPE: 0.150103
[347]	valid_0's l1: 830.121	valid_0's RMSPE: 0.150107
[348]	valid_0's l1: 829.937	valid_0's RMSPE: 0.150063
[349]	valid_0's l1: 829.625	valid_0's RMSPE: 0.150007
[350]	valid_0's l1: 829.542	valid_0's RMSPE: 0.149931
[351]	valid_0's l1: 829.302	valid_0's RMSPE: 0.149881
[352]	valid_0's l1: 829.178	valid_0's RMSPE: 0.149845
[353]	valid_0's l1: 829.129	valid_0's RMSPE: 0.149843
[354]	valid_0's l1: 828.518	valid_0's RMSPE: 0.149428
[355]	valid_0's l1: 828.469	valid_0's RMSPE: 0.149411
[356]	valid_0's l1: 828.33	valid_0's RMSPE: 0.149332
[357]	valid_0's l1: 828.214	valid_0's RMSPE: 0.149304
[358]	valid_0's l1: 828.006	valid_0's RMSPE: 0.149287
[359]	valid_0's l1: 827.823	val

[510]	valid_0's l1: 816.237	valid_0's RMSPE: 0.1438
[511]	valid_0's l1: 816.243	valid_0's RMSPE: 0.143798
[512]	valid_0's l1: 816.243	valid_0's RMSPE: 0.143798
[513]	valid_0's l1: 816.112	valid_0's RMSPE: 0.143765
[514]	valid_0's l1: 816.08	valid_0's RMSPE: 0.143748
[515]	valid_0's l1: 816.285	valid_0's RMSPE: 0.143771
[516]	valid_0's l1: 816.272	valid_0's RMSPE: 0.143769
[517]	valid_0's l1: 816.229	valid_0's RMSPE: 0.143765
[518]	valid_0's l1: 816.201	valid_0's RMSPE: 0.143748
[519]	valid_0's l1: 816.096	valid_0's RMSPE: 0.143716
[520]	valid_0's l1: 816.052	valid_0's RMSPE: 0.143694
[521]	valid_0's l1: 816.065	valid_0's RMSPE: 0.143694
[522]	valid_0's l1: 815.593	valid_0's RMSPE: 0.143596
[523]	valid_0's l1: 815.599	valid_0's RMSPE: 0.143596
[524]	valid_0's l1: 815.633	valid_0's RMSPE: 0.143602
[525]	valid_0's l1: 815.633	valid_0's RMSPE: 0.143599
[526]	valid_0's l1: 815.554	valid_0's RMSPE: 0.143543
[527]	valid_0's l1: 815.534	valid_0's RMSPE: 0.143541
[528]	valid_0's l1: 815.541	val

[674]	valid_0's l1: 810.218	valid_0's RMSPE: 0.142222
[675]	valid_0's l1: 810.149	valid_0's RMSPE: 0.142211
[676]	valid_0's l1: 810.116	valid_0's RMSPE: 0.142206
[677]	valid_0's l1: 809.969	valid_0's RMSPE: 0.142188
[678]	valid_0's l1: 809.881	valid_0's RMSPE: 0.142189
[679]	valid_0's l1: 809.807	valid_0's RMSPE: 0.142177
[680]	valid_0's l1: 809.794	valid_0's RMSPE: 0.142175
[681]	valid_0's l1: 809.703	valid_0's RMSPE: 0.142161
[682]	valid_0's l1: 809.7	valid_0's RMSPE: 0.142163
[683]	valid_0's l1: 809.69	valid_0's RMSPE: 0.142161
[684]	valid_0's l1: 809.657	valid_0's RMSPE: 0.142158
[685]	valid_0's l1: 809.607	valid_0's RMSPE: 0.14215
[686]	valid_0's l1: 809.591	valid_0's RMSPE: 0.142149
[687]	valid_0's l1: 809.583	valid_0's RMSPE: 0.142148
[688]	valid_0's l1: 809.501	valid_0's RMSPE: 0.142125
[689]	valid_0's l1: 809.546	valid_0's RMSPE: 0.142127
[690]	valid_0's l1: 809.527	valid_0's RMSPE: 0.142121
[691]	valid_0's l1: 809.53	valid_0's RMSPE: 0.142121
[692]	valid_0's l1: 809.517	valid

[837]	valid_0's l1: 806.868	valid_0's RMSPE: 0.141444
[838]	valid_0's l1: 806.836	valid_0's RMSPE: 0.141439
[839]	valid_0's l1: 806.799	valid_0's RMSPE: 0.141438
[840]	valid_0's l1: 806.783	valid_0's RMSPE: 0.141435
[841]	valid_0's l1: 806.779	valid_0's RMSPE: 0.141438
[842]	valid_0's l1: 806.87	valid_0's RMSPE: 0.141444
[843]	valid_0's l1: 806.887	valid_0's RMSPE: 0.141446
[844]	valid_0's l1: 806.828	valid_0's RMSPE: 0.141436
[845]	valid_0's l1: 806.838	valid_0's RMSPE: 0.141434
[846]	valid_0's l1: 806.855	valid_0's RMSPE: 0.141437
[847]	valid_0's l1: 806.793	valid_0's RMSPE: 0.141428
[848]	valid_0's l1: 806.781	valid_0's RMSPE: 0.141427
[849]	valid_0's l1: 806.691	valid_0's RMSPE: 0.141414
[850]	valid_0's l1: 806.817	valid_0's RMSPE: 0.141428
[851]	valid_0's l1: 806.825	valid_0's RMSPE: 0.14143
[852]	valid_0's l1: 806.817	valid_0's RMSPE: 0.141428
[853]	valid_0's l1: 806.877	valid_0's RMSPE: 0.141434
[854]	valid_0's l1: 806.921	valid_0's RMSPE: 0.141444
[855]	valid_0's l1: 806.892	va

In [12]:
RMSPE(model.predict(test_set.drop(columns=['Sales'])), test_set['Sales'])

0.14558071042770168